In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
malay_words = malaya.text.bahasa.cambridge_words.words | malaya.text.bahasa.kamus_dewan.words | malaya.text.bahasa.words.words
malay_words = list(malay_words)
len(malay_words)

63287

In [5]:
def crawl_dbp(word):
    url = f'https://prpm.dbp.gov.my/Cari1?keyword={word}'
    r = requests.get(url)
    soup = BeautifulSoup(r._content)
    definitions = soup.select("div[class*=tab-pane]")
    definitions = [d.text for d in definitions]
    spans = soup.find_all('span', id = 'MainContent_SearchInfoTesaurus_lblTesaurus')
    if len(spans):
        tesaurus = spans[0]
        tds = tesaurus.findAll('td')
        selected_td = None
        for td in tds:
            if 'javascript:showModalDialog' in str(td):
                selected_td = td
        if selected_td:
            tesaurus = [a.text for a in selected_td.find_all('a')]
        else:
            tesaurus = None
    else:
        tesaurus = None
    return {'definisi': definitions, 'tesaurus': tesaurus}

In [6]:
%%time

crawl_dbp('terserlah')

CPU times: user 16.7 ms, sys: 622 µs, total: 17.3 ms
Wall time: 392 ms


{'definisi': ['[ter.ser.lah]  |  ترسرلهDefinisi : menyerlah;\xa0(Kamus Dewan Edisi Keempat)',
  '[ter.ser.lah]  |  ترسرلهDefinisi : 1 kelihatan berseri (bkn kecantikan dll); kelihatan nyata bercahaya (bkn bulan dll): Kecantikannya ~ selepas disolek oleh mak andam. 2 kelihatan cemerlang dan menonjol (bkn bakat dll): Kebolehannya ~ dlm persembahan yg menarik itu.\xa0(Kamus Pelajar Edisi Kedua)'],
 'tesaurus': ['menyerlah', 'terserlah', 'menyerlahkan']}

In [7]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [8]:
import mp
import os
import json
from tqdm import tqdm
import traceback

directory = 'dbp'
os.makedirs(directory, exist_ok = True)

In [9]:
def loop(words):
    words, _ = words
    for word in tqdm(words):
        filename = f'{word}.json'
        filename = os.path.join(directory, filename)
        if os.path.exists(filename):
            # if os.path.getsize(filename) >= 1:
            continue
        try:
            r = crawl_dbp(word)
            with open(filename, 'w') as fopen:
                json.dump(r, fopen)
        except Exception as e:
            print(e)
            # print(traceback.format_exc())

In [10]:
loop((malay_words[:5], 0),)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.31it/s]


In [ ]:
mp.multiprocessing(malay_words, loop, cores = 20, returned = False)

 71%|██████████████████████████████████████████████████████████████▏                        | 2260/3164 [46:00<12:09,  1.24it/s]